- Put your password for sudo in `./sudo_password.txt`, since it will be needed for running CICFlowMeter.
- PCAP from original CICIDS2017 must be reordered by `reordercap`

In [1]:
PCAP_INPUT = "/mnt/extra/datasets/pcap/downloading/Wednesday-workingHours-Reordered.pcap"
FLOW_INPUT = "/mnt/extra/datasets/fixed-2/wednesday.csv"
OUTPUT_FOLDER = "/mnt/extra/datasets/processed"
# BASE_OUTPUT_NAME = os.path.splitext(os.path.basename(FLOW_INPUT))[0]
BASE_OUTPUT_NAME = "wednesday"
EXTRACT_FLOWS_OUTPUT_FOLDER = OUTPUT_FOLDER # "./ExtractFlowsOutput"

VALIDATION_PERCENTAGE = 0.1
TEST_PERCENTAGE = 0.1

GRACE_PERCENTAGE = 0.02
VALIDATION_PERCENTAGE += GRACE_PERCENTAGE
TEST_PERCENTAGE += GRACE_PERCENTAGE

LABEL_MAPPING = {
    'BENIGN': 'Not DoS',
    'DoS Hulk': 'DoS',
    'DoS GoldenEye': 'DoS',
    'DoS Slowloris': 'DoS',
    'DoS Slowhttptest - Attempted': 'DoS',
    'DoS Slowloris - Attempted': 'DoS',
    'DoS Slowhttptest': 'DoS',
    'DoS Hulk - Attempted': 'DoS',
    'DoS GoldenEye - Attempted': 'DoS',
}

In [2]:
import os

def extract_flows_from_pcap(filepath: str, output_name: str):
    basename = os.path.basename(filepath)
    flow_filename = f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{output_name}_flow.csv"
    index_filename = f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{output_name}_index.txt"

    os.system(f'sudo -S java -Djava.library.path="./Tools/jnetpcap-1.4.r1500" -jar ./Tools/CICFlowMeterV3-0.0.4-SNAPSHOT.jar {filepath} {EXTRACT_FLOWS_OUTPUT_FOLDER} < ./sudo_password.txt')
    
    os.rename(f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{basename}_Flow.csv", flow_filename)
    os.rename(f"{EXTRACT_FLOWS_OUTPUT_FOLDER}/{basename}_Flow.csv_index.txt", index_filename)

    return flow_filename, index_filename

# Pre-Process Dataset
- Adjust labels to make it compatible with research scope
- Remove rows that makes mapping between packets and flows hard

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [4]:
distinguishing_keys = [
    'Src IP',
    'Src Port',
    'Dst IP',
    'Dst Port',
    'Timestamp',
]

In [40]:
compiled = pd.read_csv(FLOW_INPUT, parse_dates=['Timestamp'])
compiled

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889,187889,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.322291,5.322291,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,187889,BENIGN,-1
1,2,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758,187758,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.326005,5.326005,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,187758,BENIGN,-1
2,3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882,189882,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.266429,5.266429,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,189882,BENIGN,-1
3,4,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117,190117,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.259919,5.259919,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,190117,BENIGN,-1
4,5,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.604285,1.886030e+05,0.000000e+00,188603,188603,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.302143,5.302143,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,188603,BENIGN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [41]:
compiled['Label'].value_counts(dropna=False)

Label
BENIGN                          319120
DoS Hulk                        158468
DoS GoldenEye                     7567
DoS Slowloris                     3859
DoS Slowhttptest - Attempted      3368
DoS Slowloris - Attempted         1847
DoS Slowhttptest                  1740
DoS Hulk - Attempted               581
DoS GoldenEye - Attempted           80
Heartbleed                          11
Name: count, dtype: int64

## Test Start

In [42]:
# TEST
TEST_LABEL_MAPPING = {
    'BENIGN': 'Not DoS',
    'DoS Hulk': 'DoS Hulk',
    'DoS GoldenEye': 'DoS GoldenEye',
    'DoS Slowloris': 'DoS Slowloris',
    'DoS Slowhttptest - Attempted': 'DoS Slowhttptest',
    'DoS Slowloris - Attempted': 'DoS Slowloris',
    'DoS Slowhttptest': 'DoS Slowhttptest',
    'DoS Hulk - Attempted': 'DoS Hulk',
    'DoS GoldenEye - Attempted': 'DoS GoldenEye',
}

compiled['Label'] = compiled['Label'].map(TEST_LABEL_MAPPING)
compiled = compiled.dropna(subset=['Label'])
compiled['Label'].value_counts(dropna=False)

Label
Not DoS             319120
DoS Hulk            159049
DoS GoldenEye         7647
DoS Slowloris         5706
DoS Slowhttptest      5108
Name: count, dtype: int64

In [51]:
compiled_slowloris = compiled[compiled['Label'] == 'DoS Slowloris']
compiled_slowhttptest = compiled[compiled['Label'] == 'DoS Slowhttptest']
compiled_hulk = compiled[compiled['Label'] == 'DoS Hulk']
compiled_goldeneye = compiled[compiled['Label'] == 'DoS GoldenEye']

In [ ]:
slowloris_min = pd.Timestamp("2017-07-05 09:47:00") + pd.Timedelta(hours=3)
slowloris_max = pd.Timestamp("2017-07-05 10:11:00") + pd.Timedelta(hours=3)
slowhttptest_min = pd.Timestamp("2017-07-05 10:14:00") + pd.Timedelta(hours=3)
slowhttptest_max = pd.Timestamp("2017-07-05 10:38:00") + pd.Timedelta(hours=3)
hulk_min = pd.Timestamp("2017-07-05 10:43:00") + pd.Timedelta(hours=3)
hulk_max = pd.Timestamp("2017-07-05 11:08:00") + pd.Timedelta(hours=3)
goldeneye_min = pd.Timestamp("2017-07-05 11:10:00") + pd.Timedelta(hours=3)
goldeneye_max = pd.Timestamp("2017-07-05 11:24:00") + pd.Timedelta(hours=3)

In [ ]:
import numpy as np

min_time = np.min(compiled_slowloris['Timestamp'] + pd.Timedelta(hours=-3))
max_time = np.max(compiled_slowloris['Timestamp'] + pd.Timedelta(hours=-3))

print(f"Tagged flows are from \"{min_time}\" to \"{max_time}\"")

Tagged flows are from "2017-07-05 09:48:46.211817" to "2017-07-05 10:10:34.560235"


In [ ]:
min_time = np.min(compiled_slowhttptest['Timestamp'] + pd.Timedelta(hours=-3))
max_time = np.max(compiled_slowhttptest['Timestamp'] + pd.Timedelta(hours=-3))

print(f"Tagged flows are from \"{min_time}\" to \"{max_time}\"")

Tagged flows are from "2017-07-05 10:15:37.936810" to "2017-07-05 10:37:49.330190"


In [54]:
min_time = np.min(compiled_hulk['Timestamp'] + pd.Timedelta(hours=-3))
max_time = np.max(compiled_hulk['Timestamp'] + pd.Timedelta(hours=-3))

print(f"Tagged flows are from \"{min_time}\" to \"{max_time}\"")

Tagged flows are from "2017-07-05 10:43:23.194704" to "2017-07-05 11:07:21.326171"


In [53]:
min_time = np.min(compiled_goldeneye['Timestamp'] + pd.Timedelta(hours=-3))
max_time = np.max(compiled_goldeneye['Timestamp'] + pd.Timedelta(hours=-3))

print(f"Tagged flows are from \"{min_time}\" to \"{max_time}\"")

Tagged flows are from "2017-07-05 11:10:03.231753" to "2017-07-05 11:19:49.515445"


## Test Ended

In [ ]:
compiled['Label'] = compiled['Label'].map(LABEL_MAPPING)
compiled = compiled.dropna(subset=['Label'])
compiled['Label'].value_counts(dropna=False)

Label
Not DoS    319120
DoS        177510
Name: count, dtype: int64

In [60]:
compiled_keys = compiled[distinguishing_keys]
has_duplicate = (compiled_keys.duplicated(keep=False)).any()
print(f"Has Duplicates? {has_duplicate}")

compiled_unique = compiled[~compiled_keys.duplicated(keep=False)]
compiled_unique

Has Duplicates? False


,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889,187889,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.322291,5.322291,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,187889,Not DoS,-1
1,2,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758,187758,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.326005,5.326005,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,187758,Not DoS,-1
2,3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882,189882,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.266429,5.266429,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,189882,Not DoS,-1
3,4,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117,190117,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.259919,5.259919,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,190117,Not DoS,-1
4,5,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.604285,1.886030e+05,0.000000e+00,188603,188603,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.302143,5.302143,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,188603,Not DoS,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [64]:
compiled_unique.to_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_preprocess_1.csv", index=False)

In [65]:
compiled = pd.read_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_preprocess_1.csv", parse_dates=['Timestamp'])
compiled

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889,187889,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.322291,5.322291,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,187889,Not DoS,-1
1,2,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758,187758,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.326005,5.326005,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,187758,Not DoS,-1
2,3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882,189882,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.266429,5.266429,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,189882,Not DoS,-1
3,4,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117,190117,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.259919,5.259919,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,190117,Not DoS,-1
4,5,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,10.604285,1.886030e+05,0.000000e+00,188603,188603,0,0.000000e+00,0.000000e+00,0,0,0,0.00,0.000000e+00,0,0,0,0,0,0,0,1,32,20,5.302143,5.302143,0,0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.000000,0,0,0.00,0.000000e+00,0,0,-1,-1,188603,Not DoS,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [66]:
compiled['Label'].value_counts(dropna=False)

Label
Not DoS    319120
DoS        177510
Name: count, dtype: int64

# Run CICFlowMeter on PCAP Dataset
- Get features from current version of CICFlowMeter (e.g. to fix bugs in features caused by old CICFlowMeter)

In [ ]:
rerun1_flow_file, rerun1_index_file = extract_flows_from_pcap(PCAP_INPUT, f"{BASE_OUTPUT_NAME}_rerun_1")

[sudo] password for meervix: 

cic.cs.unb.ca.ifm.Cmd You select: /mnt/extra/datasets/pcap/downloading/Wednesday-workingHours-Reordered.pcap
cic.cs.unb.ca.ifm.Cmd Out folder: /mnt/extra/datasets/processed
cic.cs.unb.ca.ifm.Cmd CICFlowMeter received 1 pcap file
Working on... Wednesday-workingHours-Reordered.pcap
Wednesday-workingHours-Reordered.pcap is done. total 497043 flows 
Packet stats: Total=13788878,Valid=13728094,Discarded=60784
-------------------------------------------------------------------------------


('/mnt/extra/datasets/processed/wednesday_rerun_1_flow.csv',
 '/mnt/extra/datasets/processed/wednesday_rerun_1_index.txt')

In [69]:
rerun1_flow_file = "/mnt/extra/datasets/processed/wednesday_rerun_1_flow.csv"
rerun1_index_file = "/mnt/extra/datasets/processed/wednesday_rerun_1_index.txt"

In [70]:
flowmeter = pd.read_csv(rerun1_flow_file, parse_dates=['Timestamp'], date_format="mixed")
flowmeter

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889.0,187889.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187889,NeedManualLabel
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758.0,187758.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187758,NeedManualLabel
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882.0,189882.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,189882,NeedManualLabel
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117.0,190117.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,190117,NeedManualLabel
4,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0.0,0.0,

In [71]:
flowmeter_keys = flowmeter[distinguishing_keys]
flowmeter = flowmeter[~flowmeter_keys.duplicated(keep=False)]
flowmeter

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889.0,187889.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187889,NeedManualLabel
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758.0,187758.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187758,NeedManualLabel
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882.0,189882.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,189882,NeedManualLabel
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117.0,190117.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,190117,NeedManualLabel
4,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0.0,0.0,

# Annotate Rerunned CSV using Dataset's CSV

In [72]:
compiled_keys = compiled[distinguishing_keys + ['Label']]
compiled[compiled_keys.duplicated(keep=False)]

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category


In [73]:
flowmeter_keys = flowmeter[distinguishing_keys]
flowmeter[flowmeter_keys.duplicated(keep=False)]

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label


In [75]:
left_merge = flowmeter_keys.reset_index().merge(compiled_keys, on=distinguishing_keys, how='left', indicator=True).set_index('index')
left_merge

,Src IP,Src Port,Dst IP,Dst Port,Timestamp,Label,_merge
index,,,,,,,
0,192.168.10.9,13793,52.197.242.182,443,2017-07-05 11:43:34.783702,Not DoS,both
1,192.168.10.9,13793,52.197.242.182,443,2017-07-05 11:43:35.484765,Not DoS,both
2,192.168.10.9,13794,54.65.28.113,443,2017-07-05 11:43:36.375217,Not DoS,both
3,192.168.10.9,13794,54.65.28.113,443,2017-07-05 11:43:37.075970,Not DoS,both
4,192.168.10.9,13796,52.197.242.182,443,2017-07-05 11:43:37.968708,Not DoS,both
...,...,...,...,...,...,...,...
497037,192.168.10.8,59942,198.51.152.183,80,2017-07-05 17:50:58.592115,Not DoS,both
497038,192.168.10.16,27998,192.168.10.3,53,2017-07-05 13:03:34.820723,Not DoS,both
497039,192.168.10.51,58030,23.208.163.130,443,2017-07-05 13:04:59.752472,Not DoS,both


In [77]:
flowmeter_ignored = left_merge[left_merge['_merge'] == 'left_only']
flowmeter_ignored_idx = flowmeter_ignored.index
flowmeter_merged = flowmeter.drop(index=flowmeter_ignored_idx)

In [78]:
flowmeter_merged['Label'] = left_merge[left_merge['_merge'] == "both"]['Label']

In [79]:
flowmeter_merged

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889.0,187889.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187889,Not DoS
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758.0,187758.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187758,Not DoS
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882.0,189882.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,189882,Not DoS
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117.0,190117.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,190117,Not DoS
4,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.

In [80]:
flowmeter_merged['Label'].value_counts()

Label
Not DoS    318809
DoS        177510
Name: count, dtype: int64

In [ ]:
flowmeter_merged.reset_index().to_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_annotated.csv", index=False)

## Load Checkpoint

In [5]:
flowmeter = pd.read_csv(f"{OUTPUT_FOLDER}/{BASE_OUTPUT_NAME}_annotated.csv", index_col=0, parse_dates=['Timestamp'], date_format="mixed")
flowmeter

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889.0,187889.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187889,Not DoS
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758.0,187758.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187758,Not DoS
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882.0,189882.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,189882,Not DoS
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117.0,190117.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,190117,Not DoS
4,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796

# Train-Test Split
## Test Split

In [6]:
flowmeter['Timestamp End'] = flowmeter['Timestamp'] + pd.to_timedelta(flowmeter['Flow Duration'], unit="us")
flowmeter

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889.0,187889.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187889,Not DoS,2017-07-05 11:43:34.971591
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758.0,187758.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187758,Not DoS,2017-07-05 11:43:35.672523
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882.0,189882.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,189882,Not DoS,2017-07-05 11:43:36.565099
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117.0,190117.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00

In [7]:
tagged = flowmeter[flowmeter['Label'] == 'DoS']
tagged

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9593,172.16.0.1-192.168.10.50-53816-80-6,172.16.0.1,53816,192.168.10.50,80,6,2017-07-05 12:48:54.567542,17072865,4,3,239.0,0.0,231.0,0.0,59.750000,114.228937,0.0,0.0,0.000000,0.000000,13.998822,0.410007,2.845478e+06,6.969245e+06,17071390.0,60.0,17072805.0,5.690935e+06,9.855842e+06,17071481.0,473.0,17072800.0,8.536400e+06,1.207038e+07,17071450.0,1350.0,2,0,0,0,0,0,136,104,0.234290,0.175717,0.0,231.0,34.142857,86.857025,7.544143e+03,0,2,0,2,6,0,0,0,0.750000,34.142857,59.750000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,4.000000,239.000000,3.000000,0.000000,29200,28960,2,0,32,0,1.415000e+03,0.000000e+00,1415.0,1415.0,17071390.0,0.000000e+00,17071390.0,17071390.0,-1,-1,0,0,0,17072865,DoS,2017-07-05 12:49:11.640407
9594,172.16.0.1-192.168.10.50-53840-80-6,172.16.0.1,53840,192.168.10.50,80,6,2017-07-05 12:48:54.570946,17069963,4,3,239.0,0.0,231.0,0.0,59.750000,114.228937,0.0,0.0,0.000000,0.000000,14.001202,0.410077,2.844994e+06,6.968335e+06,17069049.0,58.0,17069905.0,5.689968e+06,9.854632e+06,17069117.0,343.0,17069847.0,8.534924e+06,1.206916e+07,17069107.0,740.0,2,0,0,0,0,0,136,104,0.234330,0.175747,0.0,231.0,34.142857,86.857025,7.544143e+03,0,2,0,2,6,0,0,0,0.750000,34.142857,59.750000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,4.000000,239.000000,3.000000,0.000000,29200,28960,2,0,32,0,8.560000e+02,0.000000e+00,856.0,856.0,17069049.0,0.000000e+00,17069049.0,17069049.0,-1,-1,0,0,0,17069963,DoS,2017-07-05 12:49:11.640909
9595,172.16.0.1-192.168.10.50-53824-80-6,172.16.0.1,53824,192.168.10.50,80,6,2017-07-05 12:48:54.569148,17071303,4,3,239.0,0.0,231.0,0.0,59.750000,114.228937,0.0,0.0,0.000000,0.000000,14.000103,0.410045,2.845217e+06,6.968788e+06,17070196.0,45.0,17071198.0,5.690399e+06,9.855232e+06,17070241.0,399.0,17071214.0,8.535607e+06,1.206988e+07,17070301.0,913.0,2,0,0,0,0,0,136,104,0.234311,0.175734,0.0,231.0,34.142857,86.857025,7.544143e+03,0,2,0,2,6,0,0,0,0.750000,34.142857,59.750000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,4.000000,239.000000,3.000000,0.000000,29200,28960,2,0,32,0,1.002000e+03,0.000000e+00,1002.0,1002.0,17070196.0,0.000000e+00,17070196.0,17070196.0,-1,-1,0,0,0,17071303,DoS,2017-07-05 12:49:11.640451
9596,172.16.0.1-192.168.10.50-53834-80-6,172.16.0.1,53834,192.168.10.50,80,6,2017-07-05 12:48:54.570290,17070583,4,3,239.0,0.0,231.0,0.0,59.750000,114.228937,0.0,0.0,0.000000,0.000000,14.000693,0.410062,2.845097e+06,6.968642e

In [22]:
untagged = flowmeter[flowmeter['Label'] != 'DoS']
untagged

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.644583,1.878890e+05,0.000000e+00,187889.0,187889.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.322291,5.322291,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187889,Not DoS,2017-07-05 11:43:34.971591
1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.652010,1.877580e+05,0.000000e+00,187758.0,187758.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.326005,5.326005,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,187758,Not DoS,2017-07-05 11:43:35.672523
2,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.532857,1.898820e+05,0.000000e+00,189882.0,189882.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.266429,5.266429,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,189882,Not DoS,2017-07-05 11:43:36.565099
3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,10.519838,1.901170e+05,0.000000e+00,190117.0,190117.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.00,0.000000e+00,0.0,0.0,0,0,0,0,0,1,32,20,5.259919,5.259919,0.0,0.0,0.000000,0.000000,0.000000,0,1,1,0,1,0,0,0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.00,0.00,0.000000,8192,0,0,0,32,0,0.0,0.000000,0.0,0.0,0.00,0.000000e+00

In [8]:
import numpy as np

min_time = np.min(flowmeter['Timestamp'] + pd.Timedelta(hours=-3))
max_time = np.max(flowmeter['Timestamp'] + pd.Timedelta(hours=-3))

print(f"Whole flows are from \"{min_time}\" to \"{max_time}\"")

Whole flows are from "2017-07-05 08:42:42.084372" to "2017-07-05 17:10:14.557249"


In [9]:
min_time = np.min(tagged['Timestamp'] + pd.Timedelta(hours=-3))
max_time = np.max(tagged['Timestamp'] + pd.Timedelta(hours=-3))

print(f"Tagged flows are from \"{min_time}\" to \"{max_time}\"")

Tagged flows are from "2017-07-05 09:48:46.211817" to "2017-07-05 11:19:49.515445"


In [10]:
slowloris_min = pd.Timestamp("2017-07-05 09:47:00") + pd.Timedelta(hours=3)
slowloris_max = pd.Timestamp("2017-07-05 10:11:00") + pd.Timedelta(hours=3)
slowhttptest_min = pd.Timestamp("2017-07-05 10:14:00") + pd.Timedelta(hours=3)
slowhttptest_max = pd.Timestamp("2017-07-05 10:38:00") + pd.Timedelta(hours=3)
hulk_min = pd.Timestamp("2017-07-05 10:43:00") + pd.Timedelta(hours=3)
hulk_max = pd.Timestamp("2017-07-05 11:08:00") + pd.Timedelta(hours=3)
goldeneye_min = pd.Timestamp("2017-07-05 11:10:00") + pd.Timedelta(hours=3)
goldeneye_max = pd.Timestamp("2017-07-05 11:24:00") + pd.Timedelta(hours=3)

In [23]:
slowloris = tagged[(slowloris_min <= tagged['Timestamp']) & (tagged['Timestamp'] <= slowloris_max)]
slowloris_untagged = untagged[(slowloris_min <= untagged['Timestamp']) & (untagged['Timestamp'] <= slowloris_max)]
# slowloris = tagged[(slowloris_min <= tagged['Timestamp']) & (tagged['Timestamp End'] <= slowloris_max)]

In [24]:
slowhttptest = tagged[(slowhttptest_min <= tagged['Timestamp']) & (tagged['Timestamp'] <= slowhttptest_max)]
slowhttptest_untagged = untagged[(slowhttptest_min <= untagged['Timestamp']) & (untagged['Timestamp'] <= slowhttptest_max)]
# slowhttptest = tagged[(slowhttptest_min <= tagged['Timestamp']) & (tagged['Timestamp End'] <= slowhttptest_max)]

In [25]:
hulk = tagged[(hulk_min <= tagged['Timestamp']) & (tagged['Timestamp'] <= hulk_max)]
hulk_untagged = untagged[(hulk_min <= untagged['Timestamp']) & (untagged['Timestamp'] <= hulk_max)]
# hulk = tagged[(hulk_min <= tagged['Timestamp']) & (tagged['Timestamp End'] <= hulk_max)]

In [26]:
goldeneye = tagged[(goldeneye_min <= tagged['Timestamp']) & (tagged['Timestamp'] <= goldeneye_max)]
goldeneye_untagged = untagged[(goldeneye_min <= untagged['Timestamp']) & (untagged['Timestamp'] <= goldeneye_max)]
# goldeneye = tagged[(goldeneye_min <= tagged['Timestamp']) & (tagged['Timestamp End'] <= goldeneye_max)]

In [15]:
len(slowloris) + len(slowhttptest) + len(hulk) + len(goldeneye) == tagged['Label'].value_counts(dropna=False)['DoS']

True

In [ ]:
benign_test = int()
slowloris_test = int(len(slowloris) * TEST_PERCENTAGE)
slowhttptest_test = int(len(slowhttptest) * TEST_PERCENTAGE)
hulk_test = int(len(hulk) * TEST_PERCENTAGE)
goldeneye_test = int(len(goldeneye) * TEST_PERCENTAGE)

print("Test Dataset Numbers:")
print(f"Slowloris: {slowloris_test}")
print(f"Slowhttptest: {slowhttptest_test}")
print(f"Hulk: {hulk_test}")
print(f"Golden Eye: {goldeneye_test}")

Test Dataset Numbers:
Slowloris: 684
Slowhttptest: 612
Hulk: 19085
Golden Eye: 917


In [17]:
slowloris.sort_values('Timestamp')

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
25350,172.16.0.1-192.168.10.50-53418-80-6,172.16.0.1,53418,192.168.10.50,80,6,2017-07-05 12:48:46.211817,573660,5,4,231.0,483.0,231.0,0.0,46.200000,103.306341,483.0,0.0,120.750000,241.500000,1244.639682,15.688735,71707.500000,1.944356e+05,552804.0,2.0,573660.0,143415.000000,2.730363e+05,552920.0,1457.0,572756.0,190918.666667,3.229729e+05,563820.0,2.0,1,1,0,0,1,0,156,136,8.715964,6.972771,0.0,483.0,79.333333,169.560461,2.875075e+04,2,2,1,2,7,0,0,0,0.800000,79.333333,46.200000,120.750000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,29200,28960,1,1,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,573660,DoS,2017-07-05 12:48:46.785477
25351,172.16.0.1-192.168.10.50-53420-80-6,172.16.0.1,53420,192.168.10.50,80,6,2017-07-05 12:48:46.220819,564248,5,4,231.0,483.0,231.0,0.0,46.200000,103.306341,483.0,0.0,120.750000,241.500000,1265.401029,15.950433,70531.000000,1.945252e+05,551868.0,9.0,564248.0,141062.000000,2.739770e+05,551954.0,240.0,563645.0,187881.666667,3.246484e+05,562753.0,9.0,1,1,0,0,1,0,156,136,8.861352,7.089081,0.0,483.0,79.333333,169.560461,2.875075e+04,2,2,1,2,7,0,0,0,0.800000,79.333333,46.200000,120.750000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,29200,28960,1,1,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,564248,DoS,2017-07-05 12:48:46.785067
25352,172.16.0.1-192.168.10.50-53422-80-6,172.16.0.1,53422,192.168.10.50,80,6,2017-07-05 12:48:46.221774,561721,5,4,231.0,483.0,231.0,0.0,46.200000,103.306341,483.0,0.0,120.750000,241.500000,1271.093657,16.022189,70215.125000,1.943969e+05,551258.0,1.0,561721.0,140430.250000,2.739465e+05,551294.0,4.0,561128.0,187042.666667,3.234756e+05,560560.0,1.0,1,1,0,0,1,0,156,136,8.901216,7.120973,0.0,483.0,79.333333,169.560461,2.875075e+04,2,2,1,2,7,0,0,0,0.800000,79.333333,46.200000,120.750000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,29200,28960,1,1,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,-1,0,0,0,561721,DoS,2017-07-05 12:48:46.783495
25354,172.16.0.1-192.168.10.50-53424-80-6,172.16.0.1,53424,192.168.10.50,80,6,2017-07-05 12:48:46.222126,555185,5,4,231.0,483.0,231.0,0.0,46.200000,103.306341,483.0,0.0,120.750000,241.500000,1286.057801,16.210813,69398.125000,1.941009e+05,549767.0,2.0,555185.0,138796.250000,2.740166e+05,549816.0,588.0,554458.0,184819.333333,3.186851e+05,552804.0,2.0,1,1,0,0,1,0,156,136,9.006007,7.204806,0.0,483.0,79.333333,169.560461,2.875075e+04,2,2,1,2,7,0,0,0,0.800000,79.333333

In [34]:
slowloris_timestamps = slowloris.sort_values('Timestamp')['Timestamp'].unique()
for i in range(1, len(slowloris_timestamps)):
    if slowloris_timestamps[i-1] > slowloris_timestamps[i]:
        print("NOT SORTED")

cnt = -1
slowloris_test_df = None
slowloris_chosen_time = None
for tmp in slowloris_timestamps[::-1]:
    if cnt >= slowloris_test:
        break
    slowloris_test_df = slowloris[(tmp <= slowloris['Timestamp'])]
    slowloris_chosen_time = tmp
    cnt = len(slowloris_test_df)

slowloris_untagged_test_df = slowloris_untagged[(slowloris_chosen_time <= slowloris_untagged['Timestamp'])]

print(slowloris_test)
print(slowloris_chosen_time)
print(len(slowloris_test_df))
print(len(slowloris_untagged_test_df))

684
2017-07-05 13:07:53.191745
684
996


In [ ]:
slowhttptest_timestamps = slowhttptest.sort_values('Timestamp')['Timestamp'].unique()
for i in range(1, len(slowhttptest_timestamps)):
    if slowhttptest_timestamps[i-1] > slowhttptest_timestamps[i]:
        print("NOT SORTED")

cnt = -1
slowhttptest_test_df = None
slowhttptest_chosen_time = None
for tmp in slowhttptest_timestamps[::-1]:
    if cnt >= slowhttptest_test:
        break
    slowhttptest_test_df = slowhttptest[(tmp <= slowhttptest['Timestamp'])]
    slowhttptest_chosen_time = tmp
    cnt = len(slowhttptest_test_df)

slowhttptest_untagged_test_df = slowhttptest_untagged[(slowhttptest_chosen_time <= slowhttptest_untagged['Timestamp'])]

print(slowhttptest_test)
print(slowloris_chosen_time)
print(len(slowhttptest_test_df))
print(len(slowhttptest_untagged_test_df))

612
2017-07-05 13:07:53.191745
612
3249


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
22791,192.168.10.5-64.4.54.22-50754-80-6,192.168.10.5,50754,64.4.54.22,80,6,2017-07-05 13:33:33.123915,239937,6,5,7938.0,636.0,4320.0,0.0,1323.000000,1933.539552,636.0,0.0,127.200000,284.427847,3.573438e+04,45.845368,2.399370e+04,3.108589e+04,63896.0,3.0,239937.0,4.798740e+04,4.797875e+04,112142.0,3.0,128842.0,3.221050e+04,3.665834e+04,64107.0,246.0,2,1,0,0,0,0,132,112,25.006564,20.838804,0.0,4320.0,779.454545,1513.813156,2.291630e+06,0,2,0,3,10,0,0,0,0.833333,779.454545,1323.000000,127.200000,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,8192,8192,3,1,20,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,-1,-1,0,0,0,239937,Not DoS,2017-07-05 13:33:33.363852
22854,192.168.10.51-52.11.164.190-54627-443-6,192.168.10.51,54627,52.11.164.190,443,6,2017-07-05 13:33:52.582417,113484334,25,23,1422.0,9990.0,627.0,0.0,56.880000,156.974340,1656.0,0.0,434.347826,645.655172,1.005601e+02,0.422966,2.414560e+06,4.253149e+06,10027347.0,1.0,113399531.0,4.724980e+06,5.046990e+06,10111955.0,1.0,113399611.0,5.154528e+06,5.077617e+06,10112388.0,1.0,4,6,0,0,0,0,808,744,0.220295,0.202671,0.0,1656.0,237.750000,493.992657,2.440287e+05,0,2,0,10,47,0,0,0,0.920000,237.750000,56.880000,434.347826,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,2.083333,118.500000,1.916667,832.500000,29200,26847,4,10,32,0,292186.181818,619681.611071,2150382.0,84561.0,1.001686e+07,2.100232e+04,10027347.0,9959910.0,-1,-1,9,9,18,113484334,Not DoS,2017-07-05 13:35:46.066751
22855,192.168.10.12-208.46.17.81-40524-80-6,192.168.10.12,40524,208.46.17.81,80,6,2017-07-05 13:33:56.747177,111094751,25,28,20098.0,6433.0,2896.0,0.0,803.920000,1156.487567,919.0,0.0,229.750000,405.240909,2.388142e+02,0.477070,2.136438e+06,3.790874e+06,10240106.0,3.0,111094751.0,4.628948e+06,4.479524e+06,10240106.0,47.0,111050745.0,4.112991e+06,4.485490e+06,10284002.0,3.0,7,7,0,0,0,0,800,896,0.225033,0.252037,0.0,2896.0,500.584906,886.731432,7.862926e+05,0,0,0,14,53,0,0,0,1.120000,500.584906,803.920000,229.750000,0.0,0.0,0.000000,0.000000,0.000000,0.000000e+00,1.923077,1546.000000,2.153846,494.846154,1444,5565,11,7,32,0,124079.692308,95631.483320,262452.0,43806.0,8.407895e+06,2.061780e+06,10240106.0,5039426.0,-1,-1,4,4,8,724805845,Not DoS,2017-07-05 13:35:47.841928
22856,192.168.10.12-151.101.208.249-39100-443-6,192.168.10.12,39100,151.101.208.249,443,6,2017-07-05 13:33:57.325750,110

In [37]:
hulk_timestamps = hulk.sort_values('Timestamp')['Timestamp'].unique()
for i in range(1, len(hulk_timestamps)):
    if hulk_timestamps[i-1] > hulk_timestamps[i]:
        print("NOT SORTED")

cnt = -1
hulk_test_df = None
hulk_chosen_time = None
for tmp in hulk_timestamps[::-1]:
    if cnt >= hulk_test:
        break
    hulk_test_df = hulk[(tmp <= hulk['Timestamp'])]
    hulk_chosen_time = tmp
    cnt = len(hulk_test_df)

hulk_untagged_test_df = hulk_untagged[(hulk_chosen_time <= hulk_untagged['Timestamp'])]

print(hulk_test)
print(hulk_chosen_time)
print(len(hulk_test_df))
print(len(hulk_untagged_test_df))

19085
2017-07-05 13:58:42.492253
19085
6440


In [39]:
hulk_untagged_test_df

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
181769,192.168.10.14-157.240.2.35-52255-443-6,192.168.10.14,52255,157.240.2.35,443,6,2017-07-05 13:58:46.427225,116746714,6,3,138.0,138.0,46.0,0.0,23.000000,25.195238,46.0,46.0,46.000000,0.000000,2.364092e+00,0.077090,1.459334e+07,2.694246e+07,58250235.0,34617.0,116746714.0,2.334934e+07,3.185533e+07,58250235.0,85390.0,116661545.0,5.833077e+07,6.781861e+03,58335568.0,58325977.0,3,3,0,0,0,0,120,60,0.051393,0.025697,0.0,46.0,30.666667,23.000000,5.290000e+02,0,0,0,6,9,0,0,0,0.500000,30.666667,23.000000,46.000000,0.0,0.0,0.000000,0.00,0.00,0.000000e+00,3.000000,69.000000,1.500000,69.000000,1260,294,3,3,20,0,8.555000e+04,2.078894e+02,85697.0,85403.0,5.824511e+07,7.245016e+03,58250235.0,58239989.0,-1,-1,0,0,0,245318251,Not DoS,2017-07-05 14:00:43.173939
181771,192.168.10.8-172.217.12.206-52261-443-6,192.168.10.8,52261,172.217.12.206,443,6,2017-07-05 13:58:50.743075,119993094,142,230,4006.0,302111.0,436.0,0.0,28.211268,82.234557,3320.0,0.0,1313.526087,615.598865,2.551122e+03,3.100178,3.234315e+05,4.243799e+06,57936489.0,0.0,119956494.0,8.507553e+05,6.870986e+06,58012757.0,49.0,119956659.0,5.238282e+05,5.409209e+06,58057593.0,1.0,24,60,0,0,0,0,2852,4612,1.183401,1.916777,0.0,3320.0,822.895161,792.106103,6.274321e+05,0,2,0,84,371,0,0,0,1.619718,822.895161,28.211268,1313.526087,809.0,4.0,1183.843247,37243.50,26.25,1.130003e+06,71.000000,2003.000000,115.000000,151055.500000,8192,42780,24,217,20,0,2.097236e+06,2.638070e+06,3962633.0,231838.0,5.788101e+07,7.845703e+04,57936489.0,57825534.0,-1,-1,0,0,0,119993094,Not DoS,2017-07-05 14:00:50.736169
181772,192.168.10.8-199.187.193.1-52273-443-6,192.168.10.8,52273,199.187.193.1,443,6,2017-07-05 13:58:50.888194,119171552,24,27,1995.0,6637.0,610.0,0.0,83.125000,185.668080,1460.0,0.0,245.814815,507.066908,7.243339e+01,0.427954,2.383431e+06,4.147401e+06,10000709.0,1.0,119110563.0,5.178720e+06,4.876842e+06,10070603.0,4.0,119111509.0,4.581212e+06,4.896734e+06,10073060.0,1.0,6,6,0,0,0,0,492,548,0.201390,0.226564,0.0,1460.0,169.254902,395.327907,1.562842e+05,2,2,0,12,50,0,0,0,1.125000,169.254902,83.125000,245.814815,0.0,0.0,0.000000,0.00,0.00,0.000000e+00,1.846154,153.461538,2.076923,510.538462,8192,4380,17,8,20,0,3.525687e+05,1.006417e+06,3548339.0,59777.0,9.573311e+06,1.403452e+06,10000709.0,5120757.0,-1,-1,10,11,21,119171552,Not DoS,2017-07-05 14:00:50.059746
181773,192.168.10.8-93.184.216.16-52291-443-6,192.168.10.8,52291,9

In [40]:
goldeneye_timestamps = goldeneye.sort_values('Timestamp')['Timestamp'].unique()
for i in range(1, len(goldeneye_timestamps)):
    if goldeneye_timestamps[i-1] > goldeneye_timestamps[i]:
        print("NOT SORTED")

cnt = -1
goldeneye_test_df = None
goldeneye_chosen_time = None
for tmp in goldeneye_timestamps[::-1]:
    if cnt >= goldeneye_test:
        break
    goldeneye_test_df = goldeneye[(tmp <= goldeneye['Timestamp'])]
    goldeneye_chosen_time = tmp
    cnt = len(goldeneye_test_df)

goldeneye_untagged_test_df = goldeneye_untagged[(goldeneye_chosen_time <= goldeneye_untagged['Timestamp'])]

print(goldeneye_test)
print(goldeneye_chosen_time)
print(len(goldeneye_test_df))
print(len(goldeneye_untagged_test_df))
goldeneye_untagged_test_df

917
2017-07-05 14:18:58.660410
917
3256


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
194299,192.168.10.15-172.217.12.130-54770-443-6,192.168.10.15,54770,172.217.12.130,443,6,2017-07-05 14:19:25.726759,62375562,17,19,1014.0,4854.0,221.0,0.0,59.647059,78.116052,2313.0,0.0,255.473684,596.655900,9.407530e+01,0.577149,1.782159e+06,9.814174e+06,58130376.0,1.0,62375562.0,3.898473e+06,1.449592e+07,58207085.0,170.0,62287515.0,3.460418e+06,1.366622e+07,58167134.0,1.0,9,10,0,0,0,0,352,392,0.272543,0.304606,0.0,2313.0,163.000000,442.385417,195704.857143,0,2,0,19,35,0,0,0,1.117647,163.000000,59.647059,255.473684,0.0,0.0,0.000000,0.000,0.000,0.000000e+00,5.666667,338.000000,6.333333,1618.000000,8192,42780,9,11,20,0,4.157389e+06,0.000000,4157389.0,4157389.0,5.813038e+07,0.000000,58130376.0,58130376.0,-1,-1,0,0,0,62375562,Not DoS,2017-07-05 14:20:28.102321
194305,192.168.10.8-43.251.41.28-52946-443-6,192.168.10.8,52946,43.251.41.28,443,6,2017-07-05 14:19:33.279440,113153890,36,36,6843.0,8199.0,570.0,0.0,190.083333,271.915152,684.0,0.0,227.750000,326.656252,1.329340e+02,0.636302,1.593717e+06,3.538787e+06,9800588.0,1278.0,113153890.0,3.232968e+06,4.512517e+06,9800588.0,3906.0,112729974.0,3.220856e+06,4.813982e+06,10233057.0,1278.0,12,12,0,0,0,0,720,720,0.318151,0.318151,0.0,684.0,208.916667,299.012777,89408.640845,0,0,0,24,72,0,0,0,1.000000,208.916667,190.083333,227.750000,0.0,0.0,0.000000,0.000,0.000,0.000000e+00,3.272727,622.090909,3.272727,745.363636,64240,25923,24,12,20,0,4.501455e+05,5004.260592,452424.0,440020.0,9.797478e+06,1536.346777,9800588.0,9796361.0,-1,-1,0,0,0,484855338,Not DoS,2017-07-05 14:21:26.433330
194345,192.168.10.3-192.168.10.1-61169-53-17,192.168.10.3,61169,192.168.10.1,53,17,2017-07-05 14:19:24.709075,73678,1,1,42.0,111.0,42.0,42.0,42.000000,0.000000,111.0,111.0,111.000000,0.000000,2.076604e+03,27.145145,7.367800e+04,0.000000e+00,73678.0,73678.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,0,0,8,8,13.572573,13.572573,42.0,111.0,76.500000,48.790368,2380.500000,0,0,0,0,0,0,0,0,1.000000,76.500000,42.000000,111.000000,0.0,0.0,0.000000,0.000,0.000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0,0,1,1,8,0,0.000000e+00,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,0,0,0,73678,Not DoS,2017-07-05 14:19:24.782753
194376,192.168.10.3-192.168.10.1-60570-53-17,192.168.10.3,60570,192.168.10.1,53,17,2017-07-05 14:18:59.204989,119896,1,1,45.0,119.0,45.0,45.0,45.000000,0.000000,119.0,119.0,119.00

In [41]:
dos_test = pd.concat([
    slowloris_test_df,
    slowloris_untagged_test_df,
    slowhttptest_test_df,
    slowhttptest_untagged_test_df,
    hulk_test_df,
    hulk_untagged_test_df,
    goldeneye_test_df,
    goldeneye_untagged_test_df,
])
dos_test

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Bwd Act Data Pkts,Fwd Seg Size Min,Bwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Fwd TCP Retrans. Count,Bwd TCP Retrans. Count,Total TCP Retrans. Count,Total Connection Flow Time,Label,Timestamp End
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
15789,172.16.0.1-192.168.10.50-54158-80-6,172.16.0.1,54158,192.168.10.50,80,6,2017-07-05 13:08:05.206819,50,1,1,8.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,1.600000e+05,40000.000000,5.000000e+01,0.000000e+00,50.0,50.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1,0,0,0,0,0,32,32,20000.000000,20000.000000,0.0,8.0,4.000000,5.656854,32.000000,0,0,0,1,2,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,229,235,1,0,32,0,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1,-1,0,0,0,1140555218,DoS,2017-07-05 13:08:05.206869
15792,172.16.0.1-192.168.10.50-54004-80-6,172.16.0.1,54004,192.168.10.50,80,6,2017-07-05 13:07:53.645748,100001720,2,2,16.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,1.599972e-01,0.039999,3.333391e+07,5.773587e+07,100001551.0,77.0,100001643.0,1.000016e+08,0.000000e+00,100001643.0,100001643.0,100001628.0,1.000016e+08,0.000000e+00,100001628.0,100001628.0,2,0,0,0,0,0,64,64,0.020000,0.020000,0.0,8.0,4.000000,4.618802,21.333333,0,0,0,2,4,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,2.000000,16.000000,2.000000,0.0,229,235,2,0,32,0,92.000000,0.000000,92.0,92.0,1.000016e+08,0.000000,100001551.0,100001551.0,-1,-1,0,0,0,1239056384,DoS,2017-07-05 13:09:33.647468
15795,172.16.0.1-192.168.10.50-53996-80-6,172.16.0.1,53996,192.168.10.50,80,6,2017-07-05 13:07:53.645747,100001673,2,2,16.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,1.599973e-01,0.039999,3.333389e+07,5.773590e+07,100001567.0,45.0,100001612.0,1.000016e+08,0.000000e+00,100001612.0,100001612.0,100001628.0,1.000016e+08,0.000000e+00,100001628.0,100001628.0,2,0,0,0,0,0,64,64,0.020000,0.020000,0.0,8.0,4.000000,4.618802,21.333333,0,0,0,2,4,0,0,0,1.000000,4.000000,8.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,2.000000,16.000000,2.000000,0.0,229,235,2,0,32,0,45.000000,0.000000,45.0,45.0,1.000016e+08,0.000000,100001567.0,100001567.0,-1,-1,0,0,0,1239057091,DoS,2017-07-05 13:09:33.647420
15796,172.16.0.1-192.168.10.50-53990-80-6,172.16.0.1,53990,192.168.10.50,80,6,2017-07-05 13:07:53.645683,100001736,2,2,16.0,0.0,8.0,8.0,8.000000,0.000000,0.0,0.0,0.000000,0.000000,1.599972e-01,0.039999,3.333391e+07,5.773595e+07,100001640.0,47.0,100001687.0,1.000017e+08,0.000000e+00,100001687.0,100001687.0,100001689.0,1.000017e+08,0.000000e+00,100001689.0,1000016

In [43]:
dos_test.value_counts(dropna=False)

Flow ID                               Src IP        Src Port  Dst IP         Dst Port  Protocol  Timestamp                   Flow Duration  Total Fwd Packet  Total Bwd packets  Total Length of Fwd Packet  Total Length of Bwd Packet  Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  Fwd Packet Length Std  Bwd Packet Length Max  Bwd Packet Length Min  Bwd Packet Length Mean  Bwd Packet Length Std  Flow Bytes/s   Flow Packets/s  Flow IAT Mean  Flow IAT Std  Flow IAT Max  Flow IAT Min  Fwd IAT Total  Fwd IAT Mean  Fwd IAT Std   Fwd IAT Max  Fwd IAT Min  Bwd IAT Total  Bwd IAT Mean  Bwd IAT Std   Bwd IAT Max  Bwd IAT Min  Fwd PSH Flags  Bwd PSH Flags  Fwd URG Flags  Bwd URG Flags  Fwd RST Flags  Bwd RST Flags  Fwd Header Length  Bwd Header Length  Fwd Packets/s  Bwd Packets/s  Packet Length Min  Packet Length Max  Packet Length Mean  Packet Length Std  Packet Length Variance  FIN Flag Count  SYN Flag Count  RST Flag Count  PSH Flag Count  ACK Flag Count  URG Flag Count  